## Creating a General Python Framework for Elo Ratings

In this notebook, we will prototype some Python classes and functions to help us compute Elo ratings. It's easier to try out code in an interactive environment such as Jupyter notebooks. The end goal is to create a new module in our [`pracpred`](https://pypi.org/project/pracpred/) package that we can reuse to create Elo ratings for NBA and NCAA basketball games, and in other sports as well.

We will start out by examining the high-level concepts behind Elo ratings, and then explore how to express those concepts in Python. After we build the high-level skeleton, we will see how easy it is to implement the simple NBA Elo ratings from our [XXX prior post](xxx) in this framework.

We will build on this framework in future posts to develop more advanced Elo rating systems. Then we can examine which rating systems produce the best predictions.

### Information Required for Elo Ratings

To compute Elo ratings for any competition, we need to know several pieces of information:

1. What are the _ratings_ for each participant (team or player) prior to the event?
2. What is the _outcome_ of the event?
3. How much of a _surprise_ is that outcome releative to _expectations_?
4. Are there any specific _adjustments_ necessary to understand how much of a surprise the outcome was?

Let's examine these more closely, one at a time. To keep the explanation straightforward, we will refer to participants as "teams" and use basketball as a concrete example. However, the ideas in this section also apply to many different individual and team sports.

#### Prior Ratings

In order to use an Elo ratings system, we need to pick some time in the past where the ratings are assumed to be known. Then, as we get information from match results, we update the ratings. The hope is that, over time, the true strengths of the teams will be reflected in the ratings. If the ratings system works well, arbitrary or erroneous choices about the initial team strengths will be washed away over time by the update process.

In our simple NBA Elo ratings from our prior post, we ignored season-to-season trends in team strength. Instead, we reset all the team Elo ratings to 1500 at the beginning of each season, to keep things simple. In future posts, we will examine more advanced ways to reflect team strength across seasons, which hopefully will result in better predictions (particularly early in the regular season).

Once we've made our choice about how to set the initial Elo ratings, all we need to do is keep track of how the ratings evolve over time. That way, prior to any match, we can just get the most recent ratings of each team.  

#### Match Outcome

This concept is very simple. We are only looking at matches between two opponents. In some sports, a draw is possible. In basketball and most other sports, one team must win. We will just need to keep track of which team won, and whether a draw is possible or not.

The team that wins scores 1 point, and the team that loses scores 0 points. In the event of a draw, each team scores 0.5 points.

#### Surprise Relative to Expectations

The expected score of each team is just that team's win probability. The surprise is the team's score (as defined above based upon match outcome) minus the expected score.

For example, if a team had a 70% win probability prior to the match and ends up winning, the surprise is 0.3 relative to expectations. If that team loses, the surprise is -0.7.

If the competition can end in a draw, then we need to be able to compute the probability of a draw. We need to make sure that the 3 probabilities (team 1 wins, team 2 wins, and a draw) add up to 1.

#### Adjustments

The last piece of information we need to consider is whether the specifics of the match alter the win probabilities of either team, above and beyond what the ratings imply. In basketball, home court advantage is an example of such an adjustment. In our prior post, we found that an adjustment of 69 Elo rating points in favor of the home team was consistent with the historical regular season win percentages since 1996. In other words, separate from the actual team strengths, we are adjusting the win probabilities directly to reflect the fact that teams tend to play better at home.

Once you consider these sort of adjustments, there are a lot of other situations you might want to examine. For example, is home court advantage the same for regular season NBA games compared to playoff games? (We will examine this in an upcoming post.) You might also want to include factors such as which team has had more rest between games. The list of potential adjustments goes on and one.

The point is, any Elo rating framework needs to be flexible enough to permit the user to specify a match-by-match adjustment, expressed in the units of Elo rating points.

### Updating Elo Ratings

If we assume we know all of the required information, updating Elo ratings is very simple. All we need to do is:

1. Compute the surprise relative to expectations for each team
2. For each team, add the surprise for that team times a multiplier to the prior Elo rating

As a reminder, here are the formulas we used to update Elo ratings in our simple NBA system:

$$H_{new} = H_{prior} + K \times (W_H - p_H)$$

$$R_{new} = R_{prior} + K \times (W_R - p_R)$$


As we saw in the introductory post, the algebra of Elo ratings guarantees that the amount we add to the winning team's rating is equal and opposite to the rating points lost by the losing side. That is, Elo ratings are zero-sum.

Although we didn't look at draws in the introductory (since they don't apply to basketball), the result is similar for sports with games that can end in a draw. If a team was favored to win, a draw will reduce that team's rating by an amount proportional to the negative surprise, and the other team will have an equal and opposite increae in its rating. A draw is a partial loss from the perspective of the favored team, and a partial win from the perspective of the underdog.

Now, let's focus on the multiplier. In our simple NBA Elo ratings, we used the constant $K = 20$. We haven't yet examined the evidence of whether the value 20 is a good choice.

More advanced systems such as [FiveThirtyEight](https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/) actually use a much more complicated formula for the multiplier. Like many other Elo ratings systems out there, FiveThirtyEight's ratings for the [NFL](https://fivethirtyeight.com/features/introducing-nfl-elo-ratings/) and other sports incorporate the margin of victory (i.e., point differential), not just which team won. These systems incorporate the margin of victory by having the multiplier depend on the point differential and potentially other factors. 

There are pros and cons of including margin of victory, which we will examine in future posts. At this stage, the point is that a general Elo rating framework needs to incorporate the flexibility for a complicated multiplier formula.

### Building the Framework in Python

Let's put these high-level concepts together and express them in Python.

We need two basic calculations for Elo ratings:

1. The expected score, which depends on the ratings and the adjustment for a particular match.
2. The update, which depends the outcome, the expected score, and the multiplier.

Let's start by building the expected score computation.

#### Expected Score

Recall that the formula for the probability of a home team win in our simple NBA Elo ratings system is:

$$P(\textrm{Home team wins}) = \frac{1}{1 + 10^{-\frac{(H-R+A)}{400}}}$$

where $H$ is the home team Elo rating, $R$ is the road team Elo rating, and $A$ is the home court adjustment. Also, recall that you can write this in terms of odds factors as:

$$P(\textrm{Home team wins}) = \frac{ah}{r + ah}$$
where
$$h = 10^{\frac{H}{400}} \\ r = 10^{\frac{R}{400}} \\ a = 10^{\frac{A}{400}}$$

There is nothing magical about the values 10 and 400 in these formulas. They just happen to be the values that [Arpad Elo](https://en.wikipedia.org/wiki/Arpad_Elo) used in designing his original chess rating system. These choices mean that, ignoring home court advantage, a 400 point rating differential implies 10:1 odds in favor of the stronger team.

As we discussed in the introductory post, you could choose any positive numbers you want instead of 10 and 400.

Even more importantly, there is no reason that the win probabilities have to follow the [logistic function](https://en.wikipedia.org/wiki/Logistic_function) at all. In fact, in Elo's original work on chess ratings, he didn't use the logistic function. Our framework should allow for the use of a general win probability calculation, with the logistic function as a special case. That way, we can examine whether other probability distributions fit the historical data better than the logistic function.

Summarizing, the expected score calculation for a team needs to receive information about the two teams' ratings, and any adjustment such as home court advantage. We can use many different probability distributions to do the calculation, with the standard Elo logistic function as a special case.

#### Abstract Base Classes

Let's represent the expected score calculation by a Python [`class`](https://docs.python.org/3/tutorial/classes.html), which we will call `Calculator`. As of right now, the only thing we are asking of this class is that, given the two teams' ratings and some adjustment, it will return to us the win probabilities for each of the teams. If draws are possible, then the draw probability will be 1 minus the sum of the win probabilities.

If you need an introduction or a refresher, here is an [overview of objected-oriented programming in Python](https://realpython.com/python3-object-oriented-programming/).

Remember that we want to build a framework that can use different Elo ratings calculations. We want to be able to compare different systems to see which gives better predictions. It would be a real pain if we had to rewrite most of our code every time we change the Elo rating system. It would be much better to have the details of the Elo ratings calculations hidden from the rest of the code. That way, we won't break anything else in our code if we change the ratings calculations.

We can use [abstract base classes](https://docs.python.org/3/library/abc.html) to create the general framework we want. Here's a [tutorial on abstract base classes](https://dbader.org/blog/abstract-base-classes-in-python) (ABCs).

If we define `Calculator` as an abstract base class (ABC), we are essentially making a rule that every specific Elo ratings calculator has to follow. Let's see how it looks.

In [1]:
from abc import ABC, abstractmethod

In [2]:
class Calculator(ABC):
    """Abstract base class for Elo rating system win probability calculators"""
    def __init__(self, **kwargs):
        super().__init__()

    @abstractmethod
    def win_probs(self, **kwargs):
        """Computed win probabilities for a given match."""
        pass

In the above code, we are saying that `Calculator` is an ABC with one abstract method, `win_probs()`. This method takes some keyword arguments. Every type of Elo ratings calculator is going to have to implement some sort of `win_probs()` method. We don't need to know the details at this point.

#### The Standard Elo Logistic Function

Of course, we want to be able to create a working Elo system. The first example is the good old standard system that we've already used in our simple NBA Elo ratings. Let's recreate that calculation in our new, generalized framework.

In [3]:
import math

In [4]:
class LogisticCalculator(Calculator):
    """Logistic win probability calculator for Elo rating system"""
    def __init__(self, *, odds_scale=10, rating_scale=400):
        self.odds_scale = odds_scale
        self.rating_scale = rating_scale
        super().__init__()
    
    def win_probs(self, *, elo1, elo2, adjustment):
        odds_factor1 = math.pow(self.odds_scale, elo1/self.rating_scale)
        odds_factor2 = math.pow(self.odds_scale, elo2/self.rating_scale)
        adj = math.pow(self.odds_scale, adjustment/self.rating_scale)
        denom = odds_factor2 + adj*odds_factor1
        expected1 = (adj*odds_factor1) / denom
        expected2 = odds_factor2 / denom
        return expected1, expected2

By default, an instance of this `class` uses the standard values of 10 and 400 in the probability calculations. You can replace those values if you'd like. We'll see an example of that below.

`LogisticCalculator` inherits from `Calculator`, so it has to implement the `win_probs()` method. This method simply takes the two Elo ratings and an adjustment as parameters. This particular calculator is applicable to sports that don't have draws. If we were analyzing a sport with draws such as soccer, we would create a different class, also inheriting from `Calculator`, to handle that.

Let's test our our new Elo probability calculator.

In [5]:
calc = LogisticCalculator()

In [6]:
calc.win_probs(elo1=1500, elo2=1500, adjustment=0)

(0.5, 0.5)

In [7]:
calc.win_probs(elo1=1100, elo2=1500, adjustment=0)

(0.0909090909090909, 0.9090909090909091)

In [8]:
calc.win_probs(elo1=1900, elo2=1500, adjustment=0)

(0.9090909090909091, 0.09090909090909091)

In [9]:
calc.win_probs(elo1=1500, elo2=1500, adjustment=69)

(0.5980137657693436, 0.40198623423065644)

These results are all consistent with simple NBA Elo ratings from our introductory post.

Now let's look at an alternative rating scale, where a 2 point rating differential implies a doubling of the odds ratio.

In [10]:
other_calc = LogisticCalculator(odds_scale=2, rating_scale=2)

In [11]:
other_calc.win_probs(elo1=7, elo2=3, adjustment=0)

(0.8, 0.2)

#### The Multiplier

The situation is similar with the Elo multiplier. We want our general framework to handle potentially complicated formulas for the multiplier. In the simplest case, the multiplier is just a constant.

We can use abstract base classes here too. Let's define a `Multiplier` as an ABC.

In [12]:
class Multiplier(ABC):
    """Abstract base class for Elo rating system multipliers"""
    def __init__(self, **kwargs):
        super().__init__()

    @abstractmethod
    def value(self, **kwargs):
        """Value of multiplier"""
        pass

We can implement the simplest case of a constant multiplier this way:

In [13]:
class ConstantMultiplier(Multiplier):
    """Constant Elo rating system multiplier"""
    def __init__(self, *, constant):
        self.constant = constant
        
    def value(self, **kwargs):
        return self.constant

In [14]:
ConstantMultiplier(constant=20).value()

20

This may seem like a lot of machinery for a simple Elo rating system. However, as we build more advanced systems, it will make it much easier to mix and match different probability calculations and multipliers. If we didn't put this thought and planning into our framework upfront, we would probably end up breaking our code and have to rewrite it each time we change our Elo ratings system.

#### Elo Ratings Update

The last step in the calculation is the update. This is easy, once we know the outcome, the win probabilities and the multiplier.

First, let's create a simple class to represent the possible outcomes of a match, including draws.

In [15]:
from enum import Enum, auto

In [16]:
class MatchOutcome(Enum):
    DRAW = auto()
    WIN1 = auto()
    WIN2 = auto()

Of course, in basketball we will never need to represent a draw outcome, but we might as well incorporate it into our general framework.

Now, we can create a class to bring together all the parts of the Elo update. Let's call it `Updater`.

In [17]:
class Updater():
    def __init__(self, *, calculator, multiplier):
        self.calculator = calculator
        self.multiplier = multiplier
        
    def update(self, *, outcome, elo1, elo2, adjustment, **kwargs):
        prob1, prob2 = self.calculator.win_probs(elo1=elo1, elo2=elo2, adjustment=adjustment)
        if outcome == MatchOutcome.DRAW:
            score1 = 0.5
            score2 = 0.5
        elif outcome == MatchOutcome.WIN1:
            score1 = 1.0
            score2 = 0.0
        elif outcome == MatchOutcome.WIN2:
            score1 = 0.0
            score2 = 1.0
        else:
            raise ValueError('invalid outcome', outcome)
        k = self.multiplier.value(outcome=outcome, elo1=elo1, elo2=elo2, adjustment=adjustment, **kwargs)
        new_elo1 = elo1 + k*(score1 - prob1)
        new_elo2 = elo2 + k*(score2 - prob2)
        return new_elo1, new_elo2

Notice that `Updater` is not an abstract base class. You can think of ABCs as a template or a placeholder. This `Updater` class actually does something.

Still, we want to specialize the general `Updater` class to get a fully-working Elo rating system. We can build our simple NBA Elo ratings by inheriting from `Updater` like this:

In [18]:
class SimpleNBAEloPrototype(Updater):
    class GameType(Enum):
        REGULAR = auto()
        PLAYOFF = auto()

    def __init__(self, *, regular_season_hca=69, playoff_hca=93):
        super().__init__(
            calculator=LogisticCalculator(),
            multiplier=ConstantMultiplier(constant=20)
        )
        self.reg_hca = regular_season_hca
        self.post_hca = playoff_hca

    def update(self, *, game_type, home_team, outcome, elo1, elo2):
        if outcome == MatchOutcome.DRAW:
            raise ValueError('no draws in basketball')
        if game_type == self.GameType.REGULAR:
            hca = self.reg_hca
        elif game_type == self.GameType.PLAYOFF:
            hca = self.post_hca
        else:
            raise ValueError('invalid game type', game_type)
        if home_team == 1:
            adjustment = hca
        elif home_team == 2:
            adjustment = -hca
        else:
            raise ValueError('invalid home team', home_team)
        return super().update(outcome=outcome, elo1=elo1, elo2=elo2, adjustment=adjustment)

This class stores the information about home court advantage for us, although we can override the standard values if we want. The only new idea here is that home court advantage might be different in the playoffs than during the regular NBA season. We will examine this more closely in an upcoming post, and give a justification for the 93 Elo point adjustment used here.

Let's try out the new version of our simple NBA Elo ratings.

In [19]:
nba_elo = SimpleNBAEloPrototype()

In [20]:
nba_elo.update(
    game_type=SimpleNBAEloPrototype.GameType.REGULAR,
    home_team=1,
    outcome=MatchOutcome.WIN1,
    elo1=1500,
    elo2=1500,
)

(1508.039724684613, 1491.960275315387)

In [21]:
nba_elo.update(
    game_type=SimpleNBAEloPrototype.GameType.REGULAR,
    home_team=1,
    outcome=MatchOutcome.WIN2,
    elo1=1500,
    elo2=1500,
)

(1488.039724684613, 1511.960275315387)

### Putting It All Together

Everything looks good with our prototype. Since we want to reuse this general Elo framework in many other situations, let's incorporate it into a package. I've decided to include it in the latest version of the [`pracpred`](https://pypi.org/project/pracpred/) package. It works similarly to the prototype classes above. Here's how to use it.

In [22]:
from pracpred.ratings import elo

In [23]:
class SimpleNBAElo(elo.Updater):
    class GameType(Enum):
        REGULAR = auto()
        PLAYOFF = auto()

    def __init__(self, *, regular_season_hca=69, playoff_hca=93):
        super().__init__(
            calculator=elo.LogisticCalculator(),
            multiplier=elo.ConstantMultiplier(constant=20)
        )
        self.reg_hca = regular_season_hca
        self.post_hca = playoff_hca

    def update(self, *, game_type, home_team, outcome, elo1, elo2):
        if outcome == elo.MatchOutcome.DRAW:
            raise ValueError('no draws in basketball')
        if game_type == self.GameType.REGULAR:
            hca = self.reg_hca
        elif game_type == self.GameType.PLAYOFF:
            hca = self.post_hca
        else:
            raise ValueError('invalid game type', game_type)
        if home_team == 1:
            adjustment = hca
        elif home_team == 2:
            adjustment = -hca
        else:
            raise ValueError('invalid home team', home_team)
        return super().update(outcome=outcome, elo1=elo1, elo2=elo2, adjustment=adjustment)

In [24]:
nba_elo = SimpleNBAElo()

In [25]:
nba_elo.update(
    game_type=SimpleNBAElo.GameType.REGULAR,
    home_team=1,
    outcome=elo.MatchOutcome.WIN1,
    elo1=1500,
    elo2=1500,
)

(1508.039724684613, 1491.960275315387)

In [26]:
nba_elo.update(
    game_type=SimpleNBAElo.GameType.REGULAR,
    home_team=1,
    outcome=elo.MatchOutcome.WIN2,
    elo1=1500,
    elo2=1500,
)

(1488.039724684613, 1511.960275315387)

Going forward, we can just import the `elo` module and define the class above anytime we want to use our simple NBA Elo ratings system. We can also create more complex NBA Elo ratings systems by defining a new class with different calculator or multiplier parameters.